In [1]:
#from transformers import XLNetTokenizer, XLNetModel
#
#tokenizer = XLNetTokenizer.from_pretrained(r'\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\xlnet-base-cased')
#model = XLNetModel.from_pretrained(r'\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\xlnet-base-cased')

In [2]:
## Load data from a file
#with open('data.txt', 'r') as f:
#  data = f.read()
# 
## Tokenize the data
#tokenized_data = tokenizer(data)
# 
## Convert the tokenized data to a PyTorch tensor
#input_ids = torch.tensor(tokenized_data['input_ids'])
#attention_mask = torch.tensor(tokenized_data['attention_mask'])
# 
## Pass the input data to the model
#outputs = model(input_ids, attention_mask=attention_mask)
# 
## Get the logits from the model outputs
#logits = outputs[0]
# 
## Calculate the predicted probabilities
#probabilities = F.softmax(logits, dim=-1)
# 
## Get the predicted labels
#predictions = torch.argmax(probabilities, dim=-1)
 

In [3]:
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams
import spacy
import re

from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [4]:
df=pd.read_excel(r"DATA.xlsx")

In [5]:
df.head(5)

Grievance            Label
0  — diego the called because she is not ok with ...  Benefit Package
1  ci for assistance he does nt agree with a copa...  Benefit Package
2  tamara ticket ticket 2122024 16808 is really u...  Benefit Package
3  added email address reviewed claim port resp c...  Benefit Package
4  apologized to and filed greivance and agent ar...  Benefit Package

In [6]:
df['Label'] = df['Label'].map({'Access': 0,
'Appearance of Site':1,
'Benefit Package':2,
'CMS issue':3,
'Enrollment / Disenrollment':4,
'Humana Customer Service':5,
'Marketing':6,
'Other':7,
'Provider Customer Service':8,
'Quality of Care':9})

df = df.dropna(subset=['Label'])
df['Label'] = df['Label'].astype(int)

In [7]:
df['Label'].value_counts()

Label
6    107
7    102
2    101
8    101
0     99
5     98
4     93
9     90
1     57
3     46
Name: count, dtype: int64

In [8]:
## Preprocessing steps(Lower the cases)
df['Grievance']=df['Grievance'].apply(str.lower).str.strip()
nlp = spacy.load("en_core_web_lg")

In [9]:
def process_sentence(sentence):
    
    doc = nlp(sentence)  # Process the sentence with SpaCy
    cleaned_tokens = []   # Initialize an empty list to store modified tokens
    for token in doc:
        # Check if the token is not a name, organization, or location
        if token.ent_type_ not in ['PERSON', 'ORG', 'GPE']:
            # If not, add the token text to the cleaned_tokens list
            cleaned_tokens.append(token.text)
    cleaned_sentence = ' '.join(cleaned_tokens)  # Join the cleaned tokens back into a string
    return cleaned_sentence
   
   
def preprocess_text(text):
# Convert text to lowercase and strip whitespace
    text = text.lower().strip()
    # Remove hyperlinks
    text = re.sub(r'<a\s+(?:[^>]*?\s+)?href=(["\'])(.*?)\1[^>]*>(.*?)</a>', r'\3', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z ]', '', text)   
    text=text.strip()
    # Apply SpaCy processing to the text
    cleaned_text = process_sentence(text)
   
    return cleaned_text

In [10]:
df['Grievance'] = df['Grievance'].apply(preprocess_text)

In [11]:
df.to_csv(r"Content_Train_Data.csv")

In [12]:
df.shape

(894, 2)

In [13]:
df.head()

Grievance  Label
0  the called because she is not ok with the memb...      2
1  ci for assistance he does nt agree with a copa...      2
2  tamara ticket ticket    is really upset that s...      2
3  added email address reviewed claim port resp c...      2
4  apologized to and filed greivance and agent ar...      2

In [14]:
device = torch.device('cpu')

In [15]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification

tokenizer = XLNetTokenizer.from_pretrained(r'\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained(r'\\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\xlnet-base-cased', num_labels = 10)
model = model.to(device)

C:\Users\KPA2877\.conda\envs\dlenvl\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at \\rsc.humad.com\rdrive\FinRec\FRSolutions\FRS Reporting\RXS Information\Financial Reporting\Forecast Research Dashboard\Data Science\Member_grievance\Abhirup\Model\xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
class CustomDataset(Dataset):

    def __init__(self, Grievances, targets, tokenizer, max_len):
        self.Grievances = Grievances
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.Grievances)
    
    def __getitem__(self, item):
        Grievance = str(self.Grievances[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        Grievance,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'Grievance_text': Grievance,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [17]:
df_train, df_test = train_test_split(df, test_size=0.5, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)

In [18]:
df_train.shape, df_val.shape, df_test.shape

((447, 2), (223, 2), (224, 2))

In [19]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = CustomDataset(
    Grievances=df.Grievance.to_numpy(),
    targets=df.Label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [20]:
BATCH_SIZE = 32
MAX_LEN = 128

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [21]:
EPOCHS = 1
BATCH_SIZE = 32

param_optimizer = list(model.named_parameters())
no_decay = ['bias']#, 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01}#,
                                #{'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

C:\Users\KPA2877\.conda\envs\dlenvl\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(4,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [23]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(4,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'xlnet_model.bin')
        best_accuracy = val_acc

Epoch 1/1
----------


In [ ]:
test_acc, test_loss = eval_model(
  model,
  test_data_loader,
  device,
  len(df_test)
)

print('Test Accuracy :', test_acc)
print('Test Loss :', test_loss)